#  Data preprocessing on Yelp data set

In [5]:
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from pandasql import sqldf

## Reading the data from business dataset

In [6]:
#reading json file-yelp_academic_dataset_business.json
yelp_academic_dataset_business_json_path = '/Users/swapnakotha/Desktop/CMPE-257/yelp_dataset/yelp_academic_dataset_business.json'
yelp_business_dataset_json = pd.read_json(yelp_academic_dataset_business_json_path, lines=True)
#printing the overview of the business dataset
print(yelp_business_dataset_json.shape)
print('No of records in business dataset',yelp_business_dataset_json.shape[0])
print('No of features in business dataset',yelp_business_dataset_json.shape[1])
yelp_business_dataset_json.head()

(150346, 14)
No of records in business dataset 150346
No of features in business dataset 14


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [7]:
# 1 = open, 0 = closed
# There are significant amount of businesses that are not open anymore,
yelp_business_dataset_json.is_open.value_counts()


1    119698
0     30648
Name: is_open, dtype: int64

In [8]:
df_categories =yelp_business_dataset_json.assign(categories = yelp_business_dataset_json.categories.str.split(', ')).explode('categories')
df_categories.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
683,HxlFnUI9-RNKlKD2zUwzWQ,Skippack Sweet Spot,4019 Skippack Pike,Skippack,PA,19474,40.223222,-75.399023,3.5,18,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...",Candy Stores,"{'Monday': '15:0-21:0', 'Wednesday': '9:0-18:0..."
129108,BXxoVfl65fOkacs_WS5CRw,Coast Dental,"10427 Ulmerton Rd, Ste B-3",Largo,FL,33771,27.896325,-82.784214,1.5,19,1,{'ByAppointmentOnly': 'False'},Orthodontists,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
61727,QgwCGkTvxvkl3qUR2t6Dqw,Skin Renew Day Spa,5220 E Southport Rd,Indianapolis,IN,46237,39.667321,-86.079612,4.5,7,1,{'BusinessAcceptsCreditCards': 'True'},Laser Hair Removal,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ..."


## Getting the categories

In [9]:
#showing the business categories 
print('Total number of categories:', len(df_categories.categories.value_counts()))
#showing the top 20 business categories
print('Top 20 categories:')
df_categories.categories.value_counts()[:20]

Total number of categories: 1311
Top 20 categories:


Restaurants                  52268
Food                         27781
Shopping                     24395
Home Services                14356
Beauty & Spas                14292
Nightlife                    12281
Health & Medical             11890
Local Services               11198
Bars                         11065
Automotive                   10773
Event Planning & Services     9895
Sandwiches                    8366
American (Traditional)        8139
Active Life                   7687
Pizza                         7093
Coffee & Tea                  6703
Fast Food                     6472
Breakfast & Brunch            6239
American (New)                6097
Hotels & Travel               5857
Name: categories, dtype: int64

In [10]:
df_categories

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},Doctors,None
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},Traditional Chinese Medicine,None
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},Naturopathic/Holistic,None
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},Acupuncture,None
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},Health & Medical,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...",Bikes,"{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."
150345,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,1,"{'WheelchairAccessible': 'True', 'BusinessAcce...",Beauty & Spas,"{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19..."
150345,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,1,"{'WheelchairAccessible': 'True', 'BusinessAcce...",Permanent Makeup,"{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19..."
150345,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,1,"{'WheelchairAccessible': 'True', 'BusinessAcce...",Piercing,"{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19..."


In [11]:
df_state = yelp_business_dataset_json.assign(categories = yelp_business_dataset_json.state.str.split(', ')).explode('state')
df_state.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3749,OUSHKQ9AaBNvX90fUFMU_Q,Champ's Dog House,163 Rte 70,Medford,NJ,08055,39.901745,-74.839359,4.5,33,1,None,[NJ],"{'Monday': '6:30-18:0', 'Tuesday': '6:30-18:0'..."
21664,ysTxtSfkEpqoHXb4bcfjXQ,Action Hardware,1812 Marsh Rd,Wilmington,DE,19810,39.803865,-75.504063,4.0,11,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...",[DE],"{'Monday': '8:0-16:0', 'Tuesday': '8:0-20:0', ..."
75245,ub_cFc5t2sgFSG-Ug2K7Qw,ALMA Rest And Bar,2223 N Westshore Blvd,Tampa,FL,33607,27.965889,-82.520836,3.5,63,1,"{'RestaurantsDelivery': 'True', 'CoatCheck': '...",[FL],"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [12]:
#printing the number of states in which the businesses are available 
print('Total number of categories:', len(df_state.state.value_counts()))

Total number of categories: 27


## Cleaning the dataset

In [13]:
#countof missing values for each attribute
df_categories.isna().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      46607
categories        103
hours           73794
dtype: int64

In [14]:
#Eliminating the missing values
df_categories_clean=df_categories.dropna()

In [15]:
df_categories_clean.isna().sum()

business_id     0
name            0
address         0
city            0
state           0
postal_code     0
latitude        0
longitude       0
stars           0
review_count    0
is_open         0
attributes      0
categories      0
hours           0
dtype: int64

### Selecting a few categories from the larger dataset and merging them with reviews json

In [16]:
#Taking the business which are classified as 'Active Life' and creating a smaller subset
active_life = df_categories_clean[df_categories_clean['categories'].str.contains("Active Life")]
active_life

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
30,fvWn8oXXwbj2l79cochZyw,Altitude Trampoline Park - Boise,1301 N Milwaukee St,Boise,ID,83704,43.616763,-116.285382,5.0,30,1,"{'BusinessParking': '{'garage': False, 'street...",Active Life,"{'Monday': '10:0-20:0', 'Tuesday': '10:0-20:0'..."
38,LcAozWCMLGjwRbokaJAKMg,Edwardsville Children's Museum,722 Holyoake Rd,Edwardsville,IL,62025,38.804395,-89.949733,4.5,12,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...",Active Life,"{'Monday': '10:0-15:0', 'Tuesday': '9:30-14:0'..."
50,Hwt3_mOEmU-t--ywcemnMg,Gold's Gym,203 - 38th Ave N,St. Petersburg,FL,33704,27.808339,-82.636168,3.0,17,0,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...",Active Life,"{'Monday': '5:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
100,8KMIT0NXu30Jz5Ojo5uxaw,Cornerstone Physical Therapy Associates,"1338 Bristol Pike, Ste 203",Bensalem,PA,19020,40.072204,-74.964420,5.0,6,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",Active Life,"{'Monday': '9:0-19:0', 'Tuesday': '9:0-15:30',..."
109,I6DCYks9lqZeoZiVzW7PmA,Its Sold Here,94 York Rd,Willow Grove,PA,19090,40.142849,-75.114379,1.5,10,1,"{'BusinessAcceptsCreditCards': 'False', 'Resta...",Active Life,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150278,jYd7okFv6JMjIXMDjZNCDQ,Ace Golf,820 S Kings Ave,Brandon,FL,33511,27.926009,-82.295597,3.0,30,0,"{'GoodForKids': 'True', 'RestaurantsPriceRange...",Active Life,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
150285,fWeWzB9STxcX40AgSEQVcw,Arizona-Sonora Desert Museum,2021 N Kinney Rd,Tucson,AZ,85743,32.243733,-111.166815,4.5,448,1,"{'GoodForKids': 'True', 'BusinessParking': '{'...",Active Life,"{'Monday': '8:30-17:0', 'Tuesday': '8:30-17:0'..."
150303,JhSByBTYY1rGstRy76YmLA,Reiki with Darren,,Santa Barbara,CA,93105,34.420831,-119.698190,5.0,7,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",Active Life,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,PA,19103,39.945966,-75.169666,4.5,39,1,"{'ByAppointmentOnly': 'True', 'GoodForKids': '...",Active Life,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


## Reading the data from Reviews dataset

In [17]:
#reading the data from yelp_academic_dataset_review
yelp_academic_dataset_review_json_path = '/Users/swapnakotha/Desktop/CMPE-257/yelp_dataset/yelp_academic_dataset_review.json'

In [18]:
#Date set which contain is very large(it contaings around 6 million reviews)
#Instead of reading it at once,read it in smaller part for
size = 1000000
review = pd.read_json(yelp_academic_dataset_review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

chunk_list = []

for chunk_review in review:
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    #Merge reviews which corresponds to the business that are categorized as 'Active_life'
    chunk_merged = pd.merge(active_life, chunk_review, on='business_id', how='inner')
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
active_life_review = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

26386 out of 1,000,000 related reviews
27572 out of 1,000,000 related reviews
25482 out of 1,000,000 related reviews
22872 out of 1,000,000 related reviews
23464 out of 1,000,000 related reviews
26750 out of 1,000,000 related reviews
23804 out of 1,000,000 related reviews


In [19]:
#Converting into CSV file
active_life_review.to_csv("active_life_reviews.csv",index=False)

In [20]:
#Taking the business which are classified as 'Restaurants' and creating a smaller subset
restaurants = df_categories_clean[df_categories_clean['categories'].str.contains("Restaurants")]

In [21]:
restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",Restaurants,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...",Restaurants,"{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,"{'RestaurantsReservations': 'False', 'Restaura...",Restaurants,"{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...",Restaurants,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...",Restaurants,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [22]:
#Date set which contain is very large(it contaings around 6 million reviews)
#Instead of reading it at once,read it in smaller part for
size = 1000000
review = pd.read_json(yelp_academic_dataset_review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [ ]:
restaurant_chunks = []
for chunk_review in review:
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Merge reviews which corresponds to the business that are categorized as 'Restaurants'
    chunk_merged = pd.merge(restaurants, chunk_review, on='business_id', how='inner')
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    restaurant_chunks.append(chunk_merged)
restaurant = pd.concat(restaurant_chunks, ignore_index=True, join='outer', axis=0)

666750 out of 1,000,000 related reviews


In [ ]:
restaurant

In [ ]:
#converting it into 
restaurant.to_csv("restaurants_reviews.csv",index=False)

In [ ]:
#Taking the business which are classified as 'medical' and creating a smaller subset
medical = df_categories_clean[df_categories_clean['categories'].str.contains("Health & Medical")]
medical

In [ ]:
#Date set which contain is very large(it contaings around 6 million reviews)
#Instead of reading it at once,read it in smaller part for
size = 1000000
review = pd.read_json(yelp_academic_dataset_review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [ ]:
medical_chunks = []
for chunk_review in review:
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    #Merge reviews which corresponds to the business that are categorized as 'medical'
    chunk_merged = pd.merge(medical, chunk_review, on='business_id', how='inner')
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    medical_chunks.append(chunk_merged)
medical_reviews = pd.concat(medical_chunks, ignore_index=True, join='outer', axis=0)

In [ ]:
#converting into CSV file
medical_reviews.to_csv("medical_reviews.csv",index=False)

In [ ]:
#Taking the business which are classified as 'Home Services' and creating a smaller subset
home_services = df_categories_clean[df_categories_clean['categories'].str.contains("Home Services")]
home_services

In [ ]:
#Date set which contain is very large(it contaings around 6 million reviews)
#Instead of reading it at once,read it in smaller part for
size = 1000000
review = pd.read_json(yelp_academic_dataset_review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [ ]:
homeservices_chunks = []
for chunk_review in review:
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    #Merge reviews which corresponds to the business that are categorized as 'Home_services'
    chunk_merged = pd.merge(home_services, chunk_review, on='business_id', how='inner')
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    homeservices_chunks.append(chunk_merged)
homeservices_reviews = pd.concat(homeservices_chunks, ignore_index=True, join='outer', axis=0)

In [ ]:
#converting into CSV file
homeservices_reviews.to_csv("homeservices_reviews.csv")

In [ ]:
homeservices_reviews

In [ ]:
active_life

In [ ]:
active_life_review=pd.read_csv("active_life_reviews.csv")

In [ ]:
active_life_review

## Data visualization

In [ ]:
#visual representation of distribution of review count for the business categorized as 'Active life' across the US states
fig1 = px.scatter_geo(active_life_review, locations='state', size='review_count', locationmode='USA-states', scope='usa', color='state', height=500,width=700, hover_name='state', labels={'state':'State', 'count':"Number of Complaints"})

In [ ]:
fig1.show()

In [ ]:
#reading the data of business categorized as restaurents 
restaurant=pd.read_csv("restaurants_reviews.csv")

In [ ]:
restaurant['date']=pd.to_datetime(restaurant['date'])
restaurant_testing=restaurant

In [ ]:
restaurant_testing=restaurant_testing.set_index('date')

In [ ]:
#plot to show variation of review count across the years(from 2006 to 2022)
plt.plot(restaurant_testing['text'].resample('M').count())
plt.xlabel('Year')
plt.ylabel('Number of reviews')
plt.title('Restaurant --- Number of reviews per month')
plt.show()

In [ ]:
#calculating the review distribution based on rating 
restaurant_testing_Stars = restaurant_testing['stars'].value_counts()
review_stars_percent = restaurant_testing_Stars.apply(lambda i : i / len(restaurant_testing) * 100)
review_stars_percent

In [ ]:
#plot for review distribution
ax = review_stars_percent.sort_index(ascending=False).plot(kind='bar')
for patch in ax.patches:
    x = patch.get_bbox().get_points()[:, 0]
    y = patch.get_bbox().get_points()[1, 1]
    ax.annotate('{:.1f}%'.format(y), (x.mean(), y), ha='center', va='bottom')
plt.ylabel('Percentage of reviews')
plt.xlabel('Stars');

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
q = """SELECT state, count(*) as count
       FROM restaurant_testing
       group by state
       ;"""

In [ ]:
missing = pysqldf(q)
missing

In [ ]:
bus = pd.read_csv("business_updated.csv")
bus.head()

df_categories = bus.assign(categories = bus.categories.str.split(', ')).explode('categories')
df_categories.head()

print('Total number of categories:', len(df_categories.categories.value_counts()))
print('Top 20 categories:')
abc = df_categories.categories.value_counts()[:20]
abc.shape


from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q = """SELECT categories, count(*) as count
       FROM df_categories
       group by categories
       ;"""

missing = pysqldf(q)
missing

In [ ]:
restaurant_top = missing.sort_values('count')
Top20Cat = abc[-20:]

In [ ]:
Top20Cat

In [ ]:
fig = px.bar(Top20Cat)
fig.show()

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q = """SELECT state, count(*) as count
       FROM df_categories
       group by state
       ;"""

missing = pysqldf(q)
missing

In [ ]:
restaurant_top_states = missing.sort_values('count')
Top10State = restaurant_top_states[-10:]

In [ ]:
fig = px.bar(Top10State, x = 'state', y = 'count')
fig.show()

In [ ]:
rest = pd.read_csv("restaurants_reviews.csv")
rest.head()

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

q = """SELECT name, count(*) as count
       FROM rest
       group by name
       ;"""

wordcloud1 = pysqldf(q)
wordcloud1

In [ ]:
abc = wordcloud1.sort_values('count')
abc[-20:]

df = abc[-50:]
abc = ''.join(df['name'])

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = STOPWORDS,
                min_font_size = 10).generate(abc)

plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

q = """SELECT name, count(*) as count
       FROM rest
       group by name
       ;"""

state_plot = pysqldf(q)
